<a href="https://colab.research.google.com/github/Ankitarora2/TechnoCulture/blob/main/Another_copy_of_OnboardingHelper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Serve the model using vLLM

In [1]:
!pip install dspy-ai vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.6/152.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 99.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
# Run server in foreground
# !python -m vllm.entrypoints.openai.api_server --model TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ --quantization awq

# Run server in the background
!nohup python -m vllm.entrypoints.openai.api_server --model TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ --quantization awq > server.log 2>&1 &
# stdout is redirected to a file `server.log` using `> server.log`.
# We use a quantized model prepared using AWQ quantization

In [4]:
# Run this cell again and again to monitor the status of the server.
# The server can take a few mintues to start.
# Once the server has started, you will see logs such as this:
# INFO 02-10 07:16:43 llm_engine.py:877] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%
!tail server.log

WARNING 02-15 00:09:36 serving_chat.py:264] No chat template provided. Chat API will not work.
INFO:     Started server process [2364]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO 02-15 00:09:46 llm_engine.py:877] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%
INFO 02-15 00:09:56 llm_engine.py:877] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%
INFO 02-15 00:10:06 llm_engine.py:877] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%
INFO 02-15 00:10:16 llm_engine.py:877] Avg prompt thr

In [5]:
# Once the server is up and running, this should work
!curl http://localhost:8000/v1/models

{"object":"list","data":[{"id":"TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ","object":"model","created":1707955841,"owned_by":"vllm","root":"TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ","parent":null,"permission":[{"id":"modelperm-30b5f53111054ef4b142d0d4c15e0395","object":"model_permission","created":1707955841,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

# DSPy: 𝗗eclarative 𝗦elf-improving Language 𝗣rograms

In [6]:
import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch, BootstrapFinetune

In [7]:
lm = dspy.HFClientVLLM(model="TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ", port=8000, url="http://localhost")

dspy.settings.configure(lm=lm)

In [8]:
predict = dspy.Predict('question -> answer')

predict(question="What is the capital of Germany?")

Prediction(
    answer='Berlin'
)

# Onboarding Task: Build a reasonably complex pipeline
- Implement a bunch of signatures, and modules
- Use more than one teleprompter to compile and optimize the prompt pipeline
- Important to use kNN Few Shot and Chain of thought as part of the solution
- End with an ablation study showing the importance of various parameters and modules with matplotlib plots
- Use assert and suggust from dspy to further improve your dspy programs, document improvement

> No need to use RAG for this task.

In [ ]:

train = [
    ('I have been diagnosed with pneumonia, what medication should I take?', 'pneumonia, medication'),
    ('My doctor prescribed me Amoxicillin for my strep throat, is that sufficient?', 'Amoxicillin, strep throat'),
    ('I suffer from asthma, what are the best treatments available?', 'asthma, treatments'),
    ('My grandmother has been diagnosed with Alzheimer\'s disease, how can we manage her symptoms?', 'Alzheimer\'s disease, symptoms'),
    ('What are the symptoms of diabetes and how can I manage them?', 'diabetes, symptoms, management'),
    ('I have a history of heart disease, what lifestyle changes should I make?', 'heart disease, lifestyle changes'),
    ('My child has been diagnosed with ADHD, what are our options for treatment?', 'ADHD, treatment options'),
    ('I have a family history of cancer, what preventive measures should I take?', 'cancer, preventive measures'),
    ('What are the signs of depression and how can I seek help?', 'depression, signs, seeking help'),
    ('I have been experiencing anxiety attacks, what coping mechanisms can I use?', 'anxiety attacks, coping mechanisms')
]

dev = [
    ('I think I broke my wrist, what should I do?', 'wrist, injury'),
    ('My grandmother is experiencing severe joint pain, what could be causing it?', 'joint pain, causes'),
    ('What are the early signs of Parkinson\'s disease?', 'Parkinson\'s disease, signs'),
    ('My father has high blood pressure, what diet changes should he make?', 'high blood pressure, diet changes'),
    ('I suspect I have a urinary tract infection, what are the common treatments?', 'urinary tract infection, treatments'),
    ('My sister has been diagnosed with celiac disease, what foods should she avoid?', 'celiac disease, foods to avoid'),
    ('I have been having migraines frequently, what triggers them and how can I prevent them?', 'migraines, triggers, prevention'),
    ('What are the symptoms of irritable bowel syndrome and how is it diagnosed?', 'irritable bowel syndrome, symptoms, diagnosis'),
    ('My daughter has a peanut allergy, what precautions should we take?', 'peanut allergy, precautions'),
    ('I have been experiencing chest pains, what could be the cause?', 'chest pains, causes')
]


# Convert the dataset into DSPy Examples
# Convert the dataset into DSPy Examples
trainset = [dspy.Example(question=question, answer=answer).with_inputs('question') for question, answer in train]
devset = [dspy.Example(question=question, answer=answer).with_inputs('question') for question, answer in dev]

# Print the lengths of trainset and devset
print(len(trainset), len(devset))


# Print the lengths of trainset and devset
print(len(trainset), len(devset))

# Access an example from trainset and devset
train_example = trainset[0]
dev_example = devset[0]


print(train_example.question)

10 10
10 10
I have been diagnosed with pneumonia, what medication should I take?


In [ ]:
import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch, BootstrapFinetune
from dspy.teleprompt import KNNFewShot
from dspy.predict.knn import KNN

class NER(dspy.Signature):
    """Name entity recognition(Medical related)"""
    question = dspy.InputField(desc="Patient's input")
    answer = dspy.OutputField(desc="Medical name entities")


class MedicalNER(dspy.Module):
    """NER"""

    def __init__(self):
        super().__init__()
        self.remedy_suggest = dspy.ChainOfThought(NER)

    def forward(self, question, **kwargs):
        return self.remedy_suggest(question=question)






In [ ]:
pipeline = MedicalNER()
user_input = [
    "I have a headache and a fever, my thorat also feels choke",
    "My joints ache and I have a rash also i was feeling blue today",
    # Add more user input sentences here
]
for input_sentence in user_input:
    diagnosis = pipeline.forward(input_sentence)
    print(f"Input Sentence: {input_sentence}")
    print(f"Predicted Diagnosis: {diagnosis.answer}")
    print()


Input Sentence: I have a headache and a fever, my thorat also feels choke
Predicted Diagnosis: Headache, fever, and throat discomfort.

Input Sentence: My joints ache and I have a rash also i was feeling blue today
Predicted Diagnosis: Joint ache, rash, depression



In [ ]:
import dspy

class NER(dspy.Signature):
    """Name entity recognition(Medical related)"""
    question = dspy.InputField(desc="Patient's input")
    answer = dspy.OutputField(desc="Medical name entities")

class MedicalNER:
    """NER"""

    def __init__(self):
        self.remedy_suggest = dspy.ChainOfThought(NER)

    def forward(self, question, **kwargs):
        return self.remedy_suggest(question=question)

# Usage
medical_ner = MedicalNER()
result = medical_ner.forward(question="What are the symptoms of COVID-19?")
print("NER result:", result)


NER result: Prediction(
    rationale='identify the symptoms of COVID-19. We can refer to the World Health Organization (WHO) and the Centers for Disease Control and Prevention (CDC) for accurate information. According to the WHO and CDC, the most common symptoms of COVID-19 include fever, dry cough, and tiredness. Other symptoms may include aches and pains, nasal congestion, headache, conjunctivitis, sore throat, and diarrhea.',
    answer='Fever, dry cough, tiredness, aches and pains, nasal congestion, headache, conjunctivitis, sore throat, and diarrhea.'
)


In [ ]:
%%writefile parsing.py
import argparse
import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch, BootstrapFinetune
from dspy.teleprompt import KNNFewShot
from dspy.predict.knn import KNN

class RemedySuggest(dspy.Signature):
    """Suggests home remedy for the provided symptoms(only natural methods)"""
    question = dspy.InputField(desc="Patient's input")
    answer = dspy.OutputField(desc="Suggested home remedy(less than 100 words)")

class HomeRemedyPipeline(dspy.Module):
    """HomeRemedy"""

    def __init__(self):
        super().__init__()
        self.remedy_suggest = dspy.ChainOfThought(RemedySuggest)

    def forward(self, question):
        return self.remedy_suggest(question=question)

if __name__ == "__main__":
    # Initialize argparse
    parser = argparse.ArgumentParser(description='Home Remedy Pipeline')
    parser.add_argument('--question', type=str, help="Patient's input")
    args = parser.parse_args()

    # Create an instance of HomeRemedyPipeline
    home_remedy_pipeline = HomeRemedyPipeline()

    # Call the forward method with the question argument
    result = home_remedy_pipeline.forward(args.question)

    # Print the suggested home remedy
    print("Suggested home remedy:", result.answer)



Overwriting parsing.py


In [ ]:
allopathic_medicines = [
    "Paracetamol",
    "Ibuprofen",
    "Aspirin",
    "Omeprazole",
    "Metformin",
    "Simvastatin",
    "Amlodipine",
    "Levothyroxine",
    "Lisinopril",
    "Atorvastatin",
    "Metoprolol",
    "Losartan",
    "Gabapentin",
    "Amoxicillin",
    "Azithromycin",
    "Ciprofloxacin",
    "Prednisone",
    "Albuterol",
    "Hydrochlorothiazide",
    "Pantoprazole",
    "Warfarin",
    "Fluoxetine",
    "Sertraline",
    "Citalopram",
    "Escitalopram",
    "Tramadol",
    "Codeine",
    "Morphine",
    "Metronidazole",
    "Lorazepam",
    "Diazepam",
    "Acetaminophen",
    "Furosemide",
    "Fluticasone",
    "Ranitidine",
    "Clarithromycin",
    "Cephalexin",
    "Trazodone",
    "Duloxetine",
    "Venlafaxine",
    "Cyclobenzaprine",
    "Hydrocodone",
    "Methylprednisolone",
    "Prednisolone",
    "Levofloxacin",
    "Amoxicillin/clavulanate",
    "Naproxen",
    "Diphenhydramine",
    "Cetirizine",
    "Levothyroxine sodium"
]


In [ ]:
lm = dspy.HFClientVLLM(model="TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ", port=8000, url="http://localhost")

dspy.settings.configure(lm=lm)

In [ ]:
class Pipeline:
    """Pipeline for home remedy suggestions with assertions"""

    class RemedySuggest(dspy.Signature):
        """Suggest one best home remedy for the provided symptoms(only natural methods)"""
        question = dspy.InputField(desc="Patient's input")
        answer = dspy.OutputField(desc="Suggested home remedy(less than 100 words)")

    class HomeRemedyPipelineAssertions(dspy.Module):
      """Pipeline for home remedy suggestions"""
      def __init__(self):
          super().__init__()
          self.remedy_suggest = dspy.ChainOfThought(Pipeline.RemedySuggest)

      def forward(self, question, **kwargs):
          suggested_remedy = self.remedy_suggest(question=question)

          dspy.Assert(
              not any(keyword in suggested_remedy.answer.lower() for keyword in allopathic_medicines),
              "It's recommended to stick to home remedies. Avoid suggesting medicines.",
              target_module=Pipeline.RemedySuggest
          )

          # You can add more assertions or suggestions here

          return suggested_remedy

    class HomeRemedyEvaluator:
      """Evaluate the quality of home remedy suggestions."""

    @staticmethod
    def evaluate_remedy(suggested_remedy):
        # Check if the suggested remedy contains any allopathic medicines
        contains_allopathic = any(keyword in suggested_remedy.lower() for keyword in allopathic_medicines)

        # Check if the remedy is concise (less than 100 words)
        is_concise = len(suggested_remedy.split()) <= 100

        # Check if the remedy is informative (not too brief)
        is_informative = len(suggested_remedy) >= 50

        # Evaluate overall quality based on criteria
        quality = not contains_allopathic and is_concise and is_informative

        return quality

    def __init__(self):
      self.home_remedy_pipeline = self.HomeRemedyPipelineAssertions()

    def process(self, question):
      return self.home_remedy_pipeline.forward(question=question)

# Usage
pipeline = Pipeline()
result = pipeline.process(question="What is a natural remedy for white hair?")
print("Suggested remedy:", result.answer)


Suggested remedy: One natural remedy for white hair is to use a mixture of coconut oil, lemon juice, and curd. Mix equal parts of coconut oil and curd, and add a few drops of lemon juice. Apply this mixture to your hair and scalp, leave it on for 30 minutes, and then wash it off with a mild shampoo. This remedy is rich in antioxidants and nutrients that may help in promoting hair growth and preventing hair damage. It


In [ ]:


# Usage example:
suggested_remedy = result.answer
is_quality = HomeRemedyEvaluator.evaluate_remedy(suggested_remedy)
print("Is the suggested remedy of good quality?", is_quality)


Is the suggested remedy of good quality? True


In [ ]:
%%writefile parsing.py
import argparse
import dspy
import sys

lm = dspy.HFClientVLLM(model="TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ", port=8000, url="http://localhost")

dspy.settings.configure(lm=lm)
allopathic_medicines = [
    "Paracetamol",
    "Ibuprofen",
    "Aspirin",
    "Omeprazole",
    "Metformin",
    "Simvastatin",
    "Amlodipine",
    "Levothyroxine",
    "Lisinopril",
    "Atorvastatin",
    "Metoprolol",
    "Losartan",
    "Gabapentin",
    "Amoxicillin",
    "Azithromycin",
    "Ciprofloxacin",
    "Prednisone",
    "Albuterol",
    "Hydrochlorothiazide",
    "Pantoprazole",
    "Warfarin",
    "Fluoxetine",
    "Sertraline",
    "Citalopram",
    "Escitalopram",
    "Tramadol",
    "Codeine",
    "Morphine",
    "Metronidazole",
    "Lorazepam",
    "Diazepam",
    "Acetaminophen",
    "Furosemide",
    "Fluticasone",
    "Ranitidine",
    "Clarithromycin",
    "Cephalexin",
    "Trazodone",
    "Duloxetine",
    "Venlafaxine",
    "Cyclobenzaprine",
    "Hydrocodone",
    "Methylprednisolone",
    "Prednisolone",
    "Levofloxacin",
    "Amoxicillin/clavulanate",
    "Naproxen",
    "Diphenhydramine",
    "Cetirizine",
    "Levothyroxine sodium"
]



class Pipeline:
    """Pipeline for home remedy suggestions with assertions"""

    class RemedySuggest(dspy.Signature):
        """Suggest one best home remedy for the provided symptoms (only natural methods)"""
        question = dspy.InputField(desc="Patient's input")
        answer = dspy.OutputField(desc="Suggested home remedy (less than 100 words)")

    class HomeRemedyPipelineAssertions(dspy.Module):
        """Pipeline for home remedy suggestions"""

        def __init__(self):
            super().__init__()
            self.remedy_suggest = dspy.ChainOfThought(Pipeline.RemedySuggest)

        def forward(self, question, **kwargs):
            suggested_remedy = self.remedy_suggest(question=question)

            # Add assertions or suggestions here
            dspy.Assert(
                not any(keyword in suggested_remedy.answer.lower() for keyword in allopathic_medicines),
                "It's recommended to stick to home remedies. Avoid suggesting medicines.",
                target_module=Pipeline.RemedySuggest
            )

            return suggested_remedy

    def __init__(self, model_name):
        self.home_remedy_pipeline = self.HomeRemedyPipelineAssertions()
        self.model_name = model_name

    def process(self, question):
        return self.home_remedy_pipeline.forward(question=question)

def main():
    parser = argparse.ArgumentParser(description="Home Remedy Pipeline with Assertions")
    parser.add_argument("model_name", required=True, type=str, help="Name of the model")
    parser.add_argument("--question", type=str, required=True, help="Patient's input question")
    args = parser.parse_args()

    pipeline = Pipeline(model_name=args.model_name)
    result = pipeline.process(question=args.question)
    print("Suggested remedy:", result.answer)

if __name__ == "__main__":
    main()



Overwriting parsing.py


In [ ]:
!python parsing.py --question "What is a natural remedy for white hair?"

Traceback (most recent call last):
  File "/content/parsing.py", line 108, in <module>
    main()
  File "/content/parsing.py", line 99, in main
    parser.add_argument("model_name", required=True, type=str, help="Name of the model")
  File "/usr/lib/python3.10/argparse.py", line 1424, in add_argument
    kwargs = self._get_positional_kwargs(*args, **kwargs)
  File "/usr/lib/python3.10/argparse.py", line 1540, in _get_positional_kwargs
    raise TypeError(msg)
TypeError: 'required' is an invalid argument for positionals


In [ ]:

import argparse

class Pipeline:
    """Pipeline for home remedy suggestions with assertions"""

    class RemedySuggest(dspy.Signature):
        """Suggest one best home remedy for the provided symptoms(only natural methods)"""
        question = dspy.InputField(desc="Patient's input")
        answer = dspy.OutputField(desc="Suggested home remedy(less than 100 words)")

    class HomeRemedyPipelineAssertions(dspy.Module):
        """Pipeline for home remedy suggestions"""
        def __init__(self, enable_assertions=True, enable_bootstrap_fewshot=True, enable_knn_fewshot=True):
            super().__init__()
            self.enable_assertions = enable_assertions
            self.enable_bootstrap_fewshot = enable_bootstrap_fewshot
            self.enable_knn_fewshot = enable_knn_fewshot
            self.remedy_suggest = dspy.ChainOfThought(Pipeline.RemedySuggest)

            # Initialize BootstrapFewShot and KNNFewShot modules
            if self.enable_bootstrap_fewshot:
                self.bootstrap_fewshot = BootstrapFewShot()
            if self.enable_knn_fewshot:
                self.knn_fewshot = KNNFewShot()

        def forward(self, question, **kwargs):
            suggested_remedy = self.remedy_suggest(question=question)

            # Assertion checks
            if self.enable_assertions:
                dspy.Assert(
                    not any(keyword in suggested_remedy.answer.lower() for keyword in allopathic_medicines),
                    "It's recommended to stick to home remedies. Avoid suggesting medicines.",
                    target_module=Pipeline.RemedySuggest
                )

                # You can add more assertions or suggestions here

            # BootstrapFewShot
            if self.enable_bootstrap_fewshot:
              metric = dspy.evaluate.answer_exact_match
              teleprompter = BootstrapFewShot(metric=metric)
              compiled_bs = teleprompter.compile(student=HomeRemedyPipeline(), trainset=trainset)

                # Apply BootstrapFewShot logic

            # KNNFewShot
            if self.enable_knn_fewshot:
              pass
                # Apply KNNFewShot logic

            return suggested_remedy

    class HomeRemedyEvaluator:
      """Evaluate the quality of home remedy suggestions."""
    @staticmethod
    def evaluate_remedy(suggested_remedy):


        # Check if the suggested remedy contains any allopathic medicines
        contains_allopathic = any(keyword in suggested_remedy.lower() for keyword in allopathic_medicines)

        # Check if the remedy is concise (less than 100 words)
        is_concise = len(suggested_remedy.split()) <= 100

        # Check if the remedy is informative (not too brief)
        is_informative = len(suggested_remedy) >= 50

        # Evaluate overall quality based on criteria
        quality = not contains_allopathic and is_concise and is_informative

        return quality


##
    def __init__(self, enable_assertions=True, enable_bootstrap_fewshot=True, enable_knn_fewshot=True):
        self.home_remedy_pipeline = self.HomeRemedyPipelineAssertions(
            enable_assertions=enable_assertions,
            enable_bootstrap_fewshot=enable_bootstrap_fewshot,
            enable_knn_fewshot=enable_knn_fewshot
        )

    def process(self, question):
        return self.home_remedy_pipeline.forward(question=question),

if __name__ == "__main__":
    # Create argument parser
    parser = argparse.ArgumentParser(description="Home Remedy Pipeline")

    # Add arguments
    parser.add_argument("model_name", type=str, help="Name of the model to use")
    parser.add_argument("question", type=str, help="Patient's input question")
    parser.add_argument("--disable-assertions", action="store_false", help="Disable assertion checks")
    parser.add_argument("--disable-bootstrap-fewshot", action="store_false", help="Disable BootstrapFewShot")
    parser.add_argument("--disable-knn-fewshot", action="store_false", help="Disable KNNFewShot")

    # Parse arguments
    args = parser.parse_args()

    # Process pipeline
    pipeline = Pipeline(
        enable_assertions=args.disable_assertions,
        enable_bootstrap_fewshot=args.disable_bootstrap_fewshot,
        enable_knn_fewshot=args.disable_knn_fewshot
    )
    result = pipeline.process(question=args.question)
    print("Suggested remedy:", result.answer)


Writing parsing.py


In [ ]:
# %%writefile parsing.py
import argparse
import dspy

lm = dspy.HFClientVLLM(model="TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ", port=8000, url="http://localhost")

dspy.settings.configure(lm=lm)
allopathic_medicines = [
    "Paracetamol",
    "Ibuprofen",
    "Aspirin",
    "Omeprazole",
    "Metformin",
    "Simvastatin",
    "Amlodipine",
    "Levothyroxine",
    "Lisinopril",
    "Atorvastatin",
    "Metoprolol",
    "Losartan",
    "Gabapentin",
    "Amoxicillin",
    "Azithromycin",
    "Ciprofloxacin",
    "Prednisone",
    "Albuterol",
    "Hydrochlorothiazide",
    "Pantoprazole",
    "Warfarin",
    "Fluoxetine",
    "Sertraline",
    "Citalopram",
    "Escitalopram",
    "Tramadol",
    "Codeine",
    "Morphine",
    "Metronidazole",
    "Lorazepam",
    "Diazepam",
    "Acetaminophen",
    "Furosemide",
    "Fluticasone",
    "Ranitidine",
    "Clarithromycin",
    "Cephalexin",
    "Trazodone",
    "Duloxetine",
    "Venlafaxine",
    "Cyclobenzaprine",
    "Hydrocodone",
    "Methylprednisolone",
    "Prednisolone",
    "Levofloxacin",
    "Amoxicillin/clavulanate",
    "Naproxen",
    "Diphenhydramine",
    "Cetirizine",
    "Levothyroxine sodium"
]



class Pipeline:
    """Pipeline for home remedy suggestions with assertions"""

    class RemedySuggest(dspy.Signature):
        """Suggest best home remedy for the provided symptoms(only natural methods)"""
        question = dspy.InputField(desc="Patient's input")
        answer = dspy.OutputField(desc="Suggested home remedy(less than 100 words)")

    class HomeRemedyPipelineAssertions(dspy.Module):
      """Pipeline for home remedy suggestions"""
      def __init__(self):
          super().__init__()
          self.remedy_suggest = dspy.ChainOfThought(Pipeline.RemedySuggest)

      def forward(self, question, **kwargs):
          suggested_remedy = self.remedy_suggest(question=question)

          dspy.Assert(
              not any(keyword in suggested_remedy.answer.lower() for keyword in allopathic_medicines),
              "It's recommended to stick to home remedies. Avoid suggesting medicines.",
              target_module=Pipeline.RemedySuggest
          )

          # Evaluate the quality of the suggested remedy
          is_quality = HomeRemedyEvaluator.evaluate_remedy(suggested_remedy.answer)

          return suggested_remedy, is_quality

    def __init__(self):
      self.home_remedy_pipeline = self.HomeRemedyPipelineAssertions()

    def process(self, question):
      return self.home_remedy_pipeline.forward(question=question)





class HomeRemedyEvaluator:
    """Evaluate the quality of home remedy suggestions."""

    @staticmethod
    def evaluate_remedy(suggested_remedy):
        """
        Evaluate the quality of a suggested home remedy.

        Args:
            suggested_remedy (str): The suggested home remedy.

        Returns:
            bool: True if the remedy meets the quality criteria, False otherwise.
        """
        # Check if the suggested remedy contains any allopathic medicines
        allopathic_medicines = ["aspirin", "ibuprofen", "acetaminophen", "paracetamol", "antibiotics"]
        contains_allopathic = any(keyword in suggested_remedy.lower() for keyword in allopathic_medicines)

        # Check if the remedy is concise (less than 100 words)
        is_concise = len(suggested_remedy.split()) <= 100

        # Check if the remedy is informative (not too brief)
        is_informative = len(suggested_remedy) >= 50

        # Evaluate overall quality based on criteria
        quality = not contains_allopathic and is_concise and is_informative

        return quality

# Create CLI interface
def parse_args():
    parser = argparse.ArgumentParser(description="Home Remedy Pipeline")
    parser.add_argument("--question", type=str, required=True, help="Patient's input question")
    return parser.parse_args()

# Main function
def main():
    args = parse_args()
    pipeline = Pipeline()
    suggested_remedy, is_quality = pipeline.process(question=args.question)
    print("Suggested remedy:", suggested_remedy.answer)
    print("Is the suggested remedy of good quality?", is_quality)

if __name__ == "__main__":
    main()


Overwriting parsing.py


In [ ]:
!python parsing.py --question "What is a natural remedy for blue hair?"

Suggested remedy: To remove blue hair color naturally, you can try using a mixture of baking soda and lemon juice. Mix 1 tablespoon of baking soda with 1 tablespoon of lemon juice to form a paste. Apply this paste to your hair and let it sit for 15-20 minutes. Then, rinse it out with warm water. Repeat this process 2-3 times a week until the blue color fades away. This method may take some time, but it's a natural and gentle way to remove hair color.
Is the suggested remedy of good quality? True


In [ ]:
import argparse
import dspy

class Pipeline:
    """Pipeline for home remedy suggestions with assertions"""

    class RemedySuggest(dspy.Signature):
        """Suggest one best home remedy for the provided symptoms(only natural methods)"""
        question = dspy.InputField(desc="Patient's input")
        answer = dspy.OutputField(desc="Suggested home remedy(less than 100 words)")

    class HomeRemedyPipelineAssertions(dspy.Module):
        """Pipeline for home remedy suggestions"""
        def __init__(self, enable_assertions=True, enable_bootstrap_fewshot=True, enable_knn_fewshot=True):
            super().__init__()
            self.enable_assertions = enable_assertions
            self.enable_bootstrap_fewshot = enable_bootstrap_fewshot
            self.enable_knn_fewshot = enable_knn_fewshot
            self.remedy_suggest = dspy.ChainOfThought(Pipeline.RemedySuggest)

            # Initialize BootstrapFewShot and KNNFewShot modules
            if self.enable_bootstrap_fewshot:
                self.bootstrap_fewshot = BootstrapFewShot()
            if self.enable_knn_fewshot:
                self.knn_fewshot = KNNFewShot()

        def forward(self, question, **kwargs):
            suggested_remedy = self.remedy_suggest(question=question)

            # Assertion checks
            if self.enable_assertions:
                dspy.Assert(
                    not any(keyword in suggested_remedy.answer.lower() for keyword in allopathic_medicines),
                    "It's recommended to stick to home remedies. Avoid suggesting medicines.",
                    target_module=Pipeline.RemedySuggest
                )

                # You can add more assertions or suggestions here

            # BootstrapFewShot
            if self.enable_bootstrap_fewshot:
                metric = dspy.evaluate.answer_exact_match
                teleprompter = BootstrapFewShot(metric=metric)
                compiled_bs = teleprompter.compile(student=HomeRemedyPipeline(), trainset=trainset)

                # Apply BootstrapFewShot logic

            # KNNFewShot
            if self.enable_knn_fewshot:
                pass
                # Apply KNNFewShot logic

            return suggested_remedy

    class HomeRemedyEvaluator:
        """Evaluate the quality of home remedy suggestions."""

        class Assess(dspy.Signature):
          """Answer Correctness"""
          input = dspy.InputField("patient's input")
          remedy = dspy.InputField("suggested home remedy")
          score = dspy.OutputField("correctness score(range 0 to 1)")

        with dspy.context(lm=lm):
          @staticmethod
          def metric(input):
            score = dspy.Predict(Assess)
            return score

    def __init__(self, enable_assertions=True, enable_bootstrap_fewshot=True, enable_knn_fewshot=True):
        self.home_remedy_pipeline = self.HomeRemedyPipelineAssertions(
            enable_assertions=enable_assertions,
            enable_bootstrap_fewshot=enable_bootstrap_fewshot,
            enable_knn_fewshot=enable_knn_fewshot
        )

    def process(self, question):
        return self.home_remedy_pipeline.forward(question=question)

if __name__ == "__main__":
    # Create argument parser
    parser = argparse.ArgumentParser(description="Home Remedy Pipeline")

    # Add arguments
    parser.add_argument("model_name", type=str, help="Name of the model to use")
    parser.add_argument("question", type=str, help="Patient's input question")
    parser.add_argument("--disable-assertions", action="store_false", help="Disable assertion checks")
    parser.add_argument("--disable-bootstrap-fewshot", action="store_false", help="Disable BootstrapFewShot")
    parser.add_argument("--disable-knn-fewshot", action="store_false", help="Disable KNNFewShot")

    # Parse arguments
    args = parser.parse_args()

    # Process pipeline
    pipeline = Pipeline(
        enable_assertions=args.disable_assertions,
        enable_bootstrap_fewshot=args.disable_bootstrap_fewshot,
        enable_knn_fewshot=args.disable_knn_fewshot
    )
    result = pipeline.process(question=args.question)
    print("Suggested remedy:", result.answer)


usage: colab_kernel_launcher.py [-h] [--disable-assertions] [--disable-bootstrap-fewshot]
                                [--disable-knn-fewshot]
                                model_name question
colab_kernel_launcher.py: error: the following arguments are required: question


SystemExit: 2

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [25]:
import argparse
import dspy

class Pipeline:
    """Pipeline for home remedy suggestions with assertions"""

    class RemedySuggest(dspy.Signature):
        """Suggest one best home remedy for the provided symptoms(only natural methods)"""
        question = dspy.InputField(desc="Patient's input")
        answer = dspy.OutputField(desc="Suggested home remedy(less than 100 words)")

    class HomeRemedyPipelineAssertions(dspy.Module):
        """Pipeline for home remedy suggestions"""
        def __init__(self, enable_assertions=True, enable_bootstrap_fewshot=True, enable_knn_fewshot=True):
            super().__init__()
            self.enable_assertions = enable_assertions
            self.enable_bootstrap_fewshot = enable_bootstrap_fewshot
            self.enable_knn_fewshot = enable_knn_fewshot
            self.remedy_suggest = dspy.ChainOfThought(Pipeline.RemedySuggest)

            # Initialize BootstrapFewShot and KNNFewShot modules
            if self.enable_bootstrap_fewshot:
                self.bootstrap_fewshot = BootstrapFewShot()
            if self.enable_knn_fewshot:
                self.knn_fewshot = KNNFewShot()

            # Initialize HomeRemedyEvaluator
            self.evaluator = Pipeline.HomeRemedyEvaluator()

        def forward(self, question, **kwargs):
            suggested_remedy = self.remedy_suggest(question=question)

            # Pass the suggested remedy through the evaluator to check against the metric
            metric_score = self.evaluator.evaluate_remedy(question, suggested_remedy.answer)

            # Assertion checks
            if self.enable_assertions:
                dspy.Assert(
                    not any(keyword in suggested_remedy.answer.lower() for keyword in allopathic_medicines),
                    "It's recommended to stick to home remedies. Avoid suggesting medicines.",
                    target_module=Pipeline.RemedySuggest
                )

                # You can add more assertions or suggestions here

            # BootstrapFewShot
            if self.enable_bootstrap_fewshot:
                metric = dspy.evaluate.answer_exact_match
                teleprompter = BootstrapFewShot(metric=metric)
                compiled_bs = teleprompter.compile(student=HomeRemedyPipeline(), trainset=trainset)

                # Apply BootstrapFewShot logic

            # KNNFewShot
            if self.enable_knn_fewshot:
                pass
                # Apply KNNFewShot logic

            return suggested_remedy, metric_score

    class HomeRemedyEvaluator:
        """Evaluate the quality of home remedy suggestions."""

        class Assess(dspy.Signature):
          """Answer Correctness"""
          input = dspy.InputField("patient's input")
          remedy = dspy.InputField("suggested home remedy")
          score = dspy.OutputField("correctness score(range 0 to 1)")

        with dspy.context(lm=lm):
          @staticmethod
          def evaluate_remedy(input, remedy):
            score = dspy.Predict(HomeRemedyEvaluator.Assess)(input=input, remedy=remedy)
            return score

    def __init__(self, enable_assertions=True, enable_bootstrap_fewshot=True, enable_knn_fewshot=True):
        self.home_remedy_pipeline = self.HomeRemedyPipelineAssertions(
            enable_assertions=enable_assertions,
            enable_bootstrap_fewshot=enable_bootstrap_fewshot,
            enable_knn_fewshot=enable_knn_fewshot
        )

    def process(self, question):
        return self.home_remedy_pipeline.forward(question=question)

if __name__ == "__main__":
    # Create argument parser
    parser = argparse.ArgumentParser(description="Home Remedy Pipeline")

    # Add arguments
    parser.add_argument("model_name", type=str, help="Name of the model to use")
    parser.add_argument("question", type=str, help="Patient's input question")
    parser.add_argument("--disable-assertions", action="store_false", help="Disable assertion checks")
    parser.add_argument("--disable-bootstrap-fewshot", action="store_false", help="Disable BootstrapFewShot")
    parser.add_argument("--disable-knn-fewshot", action="store_false", help="Disable KNNFewShot")

    # Parse arguments
    args = parser.parse_args()

    # Process pipeline
    pipeline = Pipeline(
        enable_assertions=args.disable_assertions,
        enable_bootstrap_fewshot=args.disable_bootstrap_fewshot,
        enable_knn_fewshot=args.disable_knn_fewshot
    )
    result, metric_score = pipeline.process(question=args.question)
    print("Suggested remedy:", result.answer)
    print("Metric Score:", metric_score)


Overwriting parsing.py


In [30]:
!python parsing.py model_name --question "What is a natural remedy for blue hair?"

Traceback (most recent call last):
  File "/content/parsing.py", line 155, in <module>
    pipeline = Pipeline(
  File "/content/parsing.py", line 131, in __init__
    self.home_remedy_pipeline = self.HomeRemedyPipelineAssertions(
  File "/content/parsing.py", line 85, in __init__
    self.knn_fewshot = KNNFewShot()
NameError: name 'KNNFewShot' is not defined
